Prédire la qualité de l’air (AQI Value) à partir des moyennes (Avg) des polluants.

In [2]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [p

1. Préparation des Données: 

a. dataset Kaggle

- Charger et parser le fichier XML
- Créer un DataFrame

In [2]:
#Importer les librairies nécessaires
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
def xml_to_dataframe(xml_file):

    # Charger et parser le fichier
    tree = ET.parse(xml_file)  # adapte le nom du fichier
    root = tree.getroot()

    data = []
    # Explorer tous les éléments station

    for country in root.findall('.//Country'):
        for state in country.findall('.//State'):
            state_name = state.attrib.get('id', None)  
            for city in state.findall('.//City'):
                city_name = city.attrib.get('id', None)
                for station in city.findall('.//Station'):
                    station_name = station.attrib.get('id', None)
                    latitude = station.attrib.get('latitude', None)
                    longitude = station.attrib.get('longitude', None)
                    
                    station_data = {
                        'Country': country.attrib.get('id', None),  
                        'State': state_name,                       
                        'City': city_name,
                        'Station': station_name,
                        'Latitude': latitude,
                        'Longitude': longitude
                    }

                    # Polluants
                    for pollutant in station.findall('Pollutant_Index'):
                        pol_name = pollutant.attrib['id']
                        avg = pollutant.attrib['Avg']
                        station_data[f"{pol_name}_Avg"] = float(avg) if avg != "NA" else None

                    # AQI
                    aqi = station.find('Air_Quality_Index')
                    if aqi is not None:
                        station_data['AQI'] = int(aqi.attrib['Value'])
                        station_data['Predominant_Pollutant'] = aqi.attrib['Predominant_Parameter']
                        data.append(station_data)


    # Créer le DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
df = xml_to_dataframe('data_aqi_cpcb.xml')

#afficher le DataFrame
print(df.head())

  Country           State               City  \
0   India  Andhra_Pradesh          Anantapur   
1   India  Andhra_Pradesh           Chittoor   
2   India  Andhra_Pradesh             Kadapa   
3   India  Andhra_Pradesh  Rajamahendravaram   
4   India  Andhra_Pradesh           Tirupati   

                                          Station    Latitude   Longitude  \
0                    Gulzarpet, Anantapur - APPCB   14.675886   77.593027   
1             Gangineni Cheruvu, Chittoor - APPCB   13.204880   79.097889   
2                 Yerramukkapalli, Kadapa - APPCB   14.465052   78.824187   
3  Anand Kala Kshetram, Rajamahendravaram - APPCB  16.9872867  81.7363176   
4                      Tirumala, Tirupati - APPCB       13.67       79.35   

   PM2.5_Avg  PM10_Avg  NO2_Avg  NH3_Avg  SO2_Avg  CO_Avg  OZONE_Avg  AQI  \
0       24.0      24.0     23.0      3.0      7.0    29.0       25.0   29   
1       19.0      22.0     14.0      2.0      9.0     4.0       35.0   35   
2       37.0     

In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                454 non-null    object 
 1   State                  454 non-null    object 
 2   City                   454 non-null    object 
 3   Station                454 non-null    object 
 4   Latitude               454 non-null    object 
 5   Longitude              454 non-null    object 
 6   PM2.5_Avg              448 non-null    float64
 7   PM10_Avg               442 non-null    float64
 8   NO2_Avg                436 non-null    float64
 9   NH3_Avg                388 non-null    float64
 10  SO2_Avg                425 non-null    float64
 11  CO_Avg                 437 non-null    float64
 12  OZONE_Avg              426 non-null    float64
 13  AQI                    454 non-null    int64  
 14  Predominant_Pollutant  454 non-null    object 
dtypes: flo

,PM2.5_Avg,PM10_Avg,NO2_Avg,NH3_Avg,SO2_Avg,CO_Avg,OZONE_Avg,AQI
count,448.000000,442.000000,436.000000,388.000000,425.000000,437.000000,426.000000,454.000000
mean,151.852679,132.361991,33.993119,6.974227,16.124706,47.048055,28.504695,161.555066
std,105.825806,76.433661,30.773215,7.368440,15.105009,27.342962,28.889912,99.428323
min,10.000000,16.000000,2.000000,1.000000,1.000000,4.000000,1.000000,15.000000
25%,63.000000,78.000000,14.000000,3.000000,7.000000,28.000000,12.000000,81.000000
50%,122.500000,117.000000,26.000000,5.000000,12.000000,41.000000,21.000000,137.000000
75%,232.500000,167.750000,44.000000,9.000000,19.000000,59.000000,35.000000,234.750000
max,416.000000,419.000000,245.000000,85.000000,126.000000,170.000000,220.000000,419.000000


In [6]:
df.shape

(454, 15)

2.	Extraire les données pertinentes (features + label)

In [27]:
#Les features (X) sont : PM2.5_Avg, PM10_Avg, NO2_Avg, NH3_Avg, SO2_Avg, CO_Avg, OZONE_Avg
# Le label (y) est : AQI (la valeur à prédire)
features_cols = ['PM2.5_Avg', 'PM10_Avg', 'NO2_Avg', 'NH3_Avg', 'SO2_Avg', 'CO_Avg', 'OZONE_Avg']
label_col = 'AQI'


3.	Nettoyer et structurer les données:

    - Convertir les données en valeurs numériques;
    - Supprimer les lignes incomplètes


In [29]:
# Convertir en float tous les polluants
for col in features_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # convertit 'NA' en NaN

# Garder que les lignes complètes 
df_clean = df.dropna(subset=features_cols + [label_col]) 


4.	Séparer les données en features (X) et cibles (y)

In [30]:
X = df_clean[features_cols]
y = df_clean[label_col]
